# Project: Predicting Stock Price & Return  
Data acquisition and initial exploration
## ver 0.1
All guidance and guidlines provided by Romeo Kienzler

### Guidance - Initial data exploration    

This task is crucial for understanding your data. Data quality is the most important driver for success in any data science project. So, this task lets you address data quality from the beginning. This includes going back to the data owners and asking them for better quality data, if applicable.

### Data Source - Technology Choice and Justification  

We want to focus on one stock in this project, which is Loblaw Companies Ltd (TSX:L). We need to get stock prices and volumns, economic indecis for at least 15 years period.

For testing the ideas, we want to find some free solutions first. If the ideas work, we can switch to commercial solutions in the future.

After a few experiments, we decided to use alphavantage api to retrieve stock prices and pandas-datareader.BankOfCanadaReader to get economic indecis.

We tried to use pytrend package to scrap google trend data, but the current verson has some dependancy issues. So we decided to directly download from google trend website.



### Enterprise Data - Technology Choice and Justification  
We don't have Enterprise data in this project.

### Streaming Analytics - Technology Choice and Justification  
The stock data can be updated every business day using the API. The economic indecis and google trend data can be updated weekly or monthly. 

Currently we collect the data manually. In the future we plan to write a python script and schedule it running on google cloud computer everyday after the market is closed.

The reason that we choose google is because it is free and easy to deploy.

#### 0. Import packages

In [3]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date


import pandas_datareader as pdr
from pandas_datareader import DataReader
from pandas_datareader.bankofcanada import BankOfCanadaReader

import requests
from time import sleep

In [4]:
#let the notebook display full length of the data columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

#### 1. Read the stock prices of Loblaw Companies Ltd, competitors and indices

In [3]:
def get_stock(symbol):
    '''
    Get stock daily adjuested close price and volume using alphavantage API
    '''
    apikey = "39ID5KS0RJQLNZGL"
    base_url = "https://www.alphavantage.co/query?"
    function = "TIME_SERIES_DAILY_ADJUSTED"
    outputsize = "full"
    
    #construct the api url
    url = base_url + "function={}&symbol={}&outputsize={}&apikey={}".format(
    function, symbol, outputsize, apikey)

    r = requests.get(url)
    
    #parse the result, change the datatypes 
    try:
        df = pd.DataFrame(r.json()['Time Series (Daily)']).transpose(
            )[['5. adjusted close','6. volume']].reset_index()
    except:
        print("Cannot get stock price")
        return None
    df.columns = ['date','price','volume']
    df['date'] = pd.to_datetime(df['date'])
    df['price'] = pd.to_numeric(df['price'])
    df['volume'] = pd.to_numeric(df['volume'])
    df = df.set_index('date')
    
    print("Successfully read stock price: {}, from {} to {}".format(symbol, df.index[-1], df.index[0]))
    return df

In [17]:
#Loblaw Companies Limited is the largest Canadian food retailer
loblaw = get_stock("TSX:L")
sleep(1) #stop for a while because we are using a free API. 
#Metro Inc., a Canadian food retailer 
metro = get_stock("TSX:MRU")
sleep(1) #stop for a while because we are using a free API. 
#Empire Company Limited, the parent company of Sobeys, another Canadian food retailer
empa = get_stock("EMP-A.TO")
sleep(1) #stop for a while because we are using a free API. 
#George Weston Limited, a Canadian food processing and distribution company
gwl = get_stock("TSE:WN")
sleep(1) #stop for a while because we are using a free API. 
#Alimentation Couche-Tard, a Candian multinational operator of convenience stores
atd = get_stock("ATD-B.TO")
sleep(1) #stop for a while because we are using a free API. 
#S&P/TSX composite index, the benchmark Canadian index representing roughly 70% of the total market capitalization of the TSE companies
tsx = get_stock("^GSPTSE")
sleep(1) #stop for a while because we are using a free API. 
#S&P500, the stock market index that measures the stock performance of 500 large companies listed on stock exchanges of the US
sp500 = get_stock("^GSPC")

Successfully read stock price: TSX:L, from 2000-01-04 00:00:00 to 2019-10-18 00:00:00
Successfully read stock price: TSX:MRU, from 2000-01-03 00:00:00 to 2019-10-18 00:00:00
Successfully read stock price: EMP-A.TO, from 2000-01-03 00:00:00 to 2019-10-18 00:00:00
Successfully read stock price: TSE:WN, from 2000-01-04 00:00:00 to 2019-10-18 00:00:00
Successfully read stock price: ATD-B.TO, from 2000-01-03 00:00:00 to 2019-10-18 00:00:00
Successfully read stock price: ^GSPTSE, from 2000-01-03 00:00:00 to 2019-10-18 00:00:00
Successfully read stock price: ^GSPC, from 2000-01-03 00:00:00 to 2019-10-18 00:00:00


#### 2. Inspect the stock data

Now we have the stocks' daily adjusted close price and volume from 2000-Jan to the 2019-Oct. DataFrame.info() and DataFrame.describe() shows that there is no N/A value in the data set. However, the price and volume could be zero in some specific days.

In [12]:
print(loblaw.info())
print(loblaw.describe())
print(loblaw.head())
print(loblaw.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5019 entries, 2019-10-18 to 2000-01-04
Data columns (total 2 columns):
price     5019 non-null float64
volume    5019 non-null int64
dtypes: float64(1), int64(1)
memory usage: 117.6 KB
None
             price        volume
count  5019.000000  5.019000e+03
mean   35.577743    5.293163e+05
std    12.862054    5.922035e+05
min    0.000000     0.000000e+00
25%    26.289250    2.512000e+05
50%    32.522400    4.141000e+05
75%    44.044900    6.444000e+05
max    75.770000    1.482520e+07
            price  volume
date                     
2019-10-18  71.22  284800
2019-10-17  71.50  528200
2019-10-16  71.62  716200
2019-10-15  72.60  446100
2019-10-11  73.01  428000
              price  volume
date                       
2000-01-10  18.3117  344400
2000-01-07  18.3689  215900
2000-01-06  18.6264  186100
2000-01-05  19.0556  745400
2000-01-04  19.4276  206500


In [13]:
loblaw.describe()

,price,volume
count,5019.000000,5.019000e+03
mean,35.577743,5.293163e+05
std,12.862054,5.922035e+05
min,0.000000,0.000000e+00
25%,26.289250,2.512000e+05
50%,32.522400,4.141000e+05
75%,44.044900,6.444000e+05
max,75.770000,1.482520e+07


In [14]:
loblaw[loblaw.price == 0]

,price,volume
date,,
2004-12-28,0.0,0
2004-12-27,0.0,0
2004-10-11,0.0,0
2004-09-06,0.0,0
2004-08-02,0.0,0
2004-07-01,0.0,0
2004-05-24,0.0,0
2004-04-09,0.0,0
2004-01-01,0.0,0


In [18]:
loblaw.to_csv("loblaw.csv")
metro.to_csv("metro.csv")
empa.to_csv("empa.csv")
gwl.to_csv("gwl.csv")
atd.to_csv("atd.csv")
tsx.to_csv("tsx.csv")
sp500.to_csv("sp500.csv")

#### 3. Get the important indices

We can find the major economical indices through the API of bank of canada.   
https://www.bankofcanada.ca/valet/docs  
  
base url = "https://www.bankofcanada.ca/valet/"

To get the list of all groups: base url + "/lists/groups"  
To get the list of all series: base url + "/lists/series"  
To get the list of all series in a group: base url + "groups/groupname"  
To get the date from a series: base url + "observations/seriesname"  

Search for the key words in the group list, get the details, and retrieve the data using series name.

We use the following indices:  
series      |   groupname           |   description             |   date      
FXUSDCAD    | FX_RATES_DAILY        |USD/CAD exchange rate  | 2017-1-1, 2019-10-11  
IEXE0102    | LEGACY_CLOSING_RATES  |USD/CAD exchange rate  | 2007-5-1, 2019-4-30  
V80691311   | CHARTED_BANK_INTEREST |prime rate             | 2000-1-1, 2019-10-11  
V41690973   | CPI_MONTHLY           |consumer price index   | 2000-1, 2019-8  
W.BCPI      | BCPI_WEEKLY           |business CPI           | 2000-1-1, 2019-9-25

CEER - Canadian-dollar Effective Exchange Rate, formerly CERI - Canadian-dollar Effective Exchange Rate Index, was downloaded seperately from Bank of Canada website.  (1999-1-1 , 2019-10-11)

https://www.bankofcanada.ca/rates/exchange/canadian-effective-exchange-rates/#Overview


In [19]:
r = requests.get("https://www.bankofcanada.ca/valet/lists/groups/json")
grouplist = pd.DataFrame(r.json()['groups']).transpose()

In [20]:
grouplist[grouplist['label'].str.contains('exchange', case=False)]

,label,link
FX_RATES_RECIPROCAL,Daily reciprocal exchange rates,https://www.bankofcanada.ca/valet/groups/FX_RATES_RECIPROCAL
FX_RATES_DAILY,Daily exchange rates,https://www.bankofcanada.ca/valet/groups/FX_RATES_DAILY
FX_RATES_MONTHLY,Monthly exchange rates,https://www.bankofcanada.ca/valet/groups/FX_RATES_MONTHLY
FX_RATES_ANNUAL,Annual exchange rates,https://www.bankofcanada.ca/valet/groups/FX_RATES_ANNUAL
SWP-USCANPFX,U.S. / CAD nominal exchange rate,https://www.bankofcanada.ca/valet/groups/SWP-USCANPFX
SAN_JABS20181014,The Impact of Surprising Monetary Policy Announcements on Exchange Rate Volatility,https://www.bankofcanada.ca/valet/groups/SAN_JABS20181014
SAN_AROO20190815,Using Exchange-Traded Funds to Measure Liquidity in the Canadian Corporate Bond Market,https://www.bankofcanada.ca/valet/groups/SAN_AROO20190815
FX_RATES_DAILY_CURRENT,Daily exchange rates,https://www.bankofcanada.ca/valet/groups/FX_RATES_DAILY_CURRENT


In [21]:
group_name = "FX_RATES_DAILY"
r = requests.get("https://www.bankofcanada.ca/valet/groups/{}/json".format(group_name))
pd.DataFrame(r.json()['groupDetails']).head()

,name,label,description,groupSeries
FXAUDCAD,FX_RATES_DAILY,Daily exchange rates,Daily average exchange rates - published once each business day by 16:30 ET. All Bank of Canada exchange rates are indicative rates only.,"{'label': 'AUD/CAD', 'link': 'https://www.bankofcanada.ca/valet/series/FXAUDCAD'}"
FXBRLCAD,FX_RATES_DAILY,Daily exchange rates,Daily average exchange rates - published once each business day by 16:30 ET. All Bank of Canada exchange rates are indicative rates only.,"{'label': 'BRL/CAD', 'link': 'https://www.bankofcanada.ca/valet/series/FXBRLCAD'}"
FXCHFCAD,FX_RATES_DAILY,Daily exchange rates,Daily average exchange rates - published once each business day by 16:30 ET. All Bank of Canada exchange rates are indicative rates only.,"{'label': 'CHF/CAD', 'link': 'https://www.bankofcanada.ca/valet/series/FXCHFCAD'}"
FXCNYCAD,FX_RATES_DAILY,Daily exchange rates,Daily average exchange rates - published once each business day by 16:30 ET. All Bank of Canada exchange rates are indicative rates only.,"{'label': 'CNY/CAD', 'link': 'https://www.bankofcanada.ca/valet/series/FXCNYCAD'}"
FXEURCAD,FX_RATES_DAILY,Daily exchange rates,Daily average exchange rates - published once each business day by 16:30 ET. All Bank of Canada exchange rates are indicative rates only.,"{'label': 'EUR/CAD', 'link': 'https://www.bankofcanada.ca/valet/series/FXEURCAD'}"


In [12]:
symbol = "W.BCPI"
start = date(2000,1,1)
end = date(2019,9,30)
reader = BankOfCanadaReader(symbol, start = start, end = end)
BCPI = reader.read()
BCPI.index.name = 'date'
BCPI.columns = ['BCPI']
print(BCPI.head())
print(BCPI.tail())

BCPI
date              
2019-09-25  432.33
2019-09-18  433.06
2019-09-11  428.60
2019-09-04  421.88
2019-08-28  417.90
              BCPI
date              
2000-02-02  305.83
2000-01-26  302.82
2000-01-19  295.11
2000-01-12  284.80
2000-01-05  284.90


In [13]:
BCPI.to_csv("BCPI.csv")

In [14]:
symbol = "V41690973"
start = date(2000,1,1)
end = date(2019,9,30)
reader = BankOfCanadaReader(symbol, start = start, end = end)
CPI = reader.read()
CPI.index.name = 'date'
CPI.columns = ['CPI']
print(CPI.head())
print(CPI.tail())

CPI
date             
2019-09-01  136.2
2019-08-01  136.8
2019-07-01  137.0
2019-06-01  136.3
2019-05-01  136.6
             CPI
date            
2000-05-01  94.9
2000-04-01  94.5
2000-03-01  94.8
2000-02-01  94.1
2000-01-01  93.5


In [15]:
CPI.to_csv("CPI.csv")

In [16]:
symbol = "V80691311"
start = date(2000,1,1)
end = date(2019,9,30)
reader = BankOfCanadaReader(symbol, start = start, end = end)
bank_interest = reader.read()
bank_interest.index.name = 'date'
bank_interest.columns = ['bank_interest']
print(bank_interest.head())
print(bank_interest.tail())

bank_interest
date                     
2019-09-25  3.95         
2019-09-18  3.95         
2019-09-11  3.95         
2019-09-04  3.95         
2019-08-28  3.95         
            bank_interest
date                     
2000-02-02  6.5          
2000-01-26  6.5          
2000-01-19  6.5          
2000-01-12  6.5          
2000-01-05  6.5          


In [17]:
bank_interest.to_csv("bank_interest.csv")

In [38]:
CEER = pd.read_csv("CEER.csv", parse_dates=['date'], index_col='date')
CEER.columns = ['CEER']
print(CEER.head())
print(CEER.tail())

CEER
date             
1999-01-01  95.43
1999-01-04  95.47
1999-01-05  95.84
1999-01-06  96.66
1999-01-07  96.46
              CEER
date              
2019-10-07  117.16
2019-10-08  117.08
2019-10-09  117.03
2019-10-10  117.15
2019-10-11  117.75


In [39]:
CEER.to_csv("CEER.csv")

#### 4. Get google trend data

Here we downloaded google trend data manually. We will change it to using pytrend package in the future.  
The keywords are: grocery store, loblaws, stock


In [18]:
trend_grocery_store = pd.read_csv("trend_grocery_store.csv", parse_dates=['Month'], index_col = 'Month')
trend_grocery_store.index.name = 'date'
trend_grocery_store.columns = ['trend_grocery_store']
print(trend_grocery_store.head())
print(trend_grocery_store.tail())

trend_grocery_store
date                           
2004-01-01  47                 
2004-02-01  44                 
2004-03-01  41                 
2004-04-01  39                 
2004-05-01  40                 
            trend_grocery_store
date                           
2019-06-01  85                 
2019-07-01  98                 
2019-08-01  100                
2019-09-01  87                 
2019-10-01  80                 


In [19]:
trend_grocery_store.to_csv("trend_grocery_store.csv")

In [20]:
trend_loblaws = pd.read_csv("trend_loblaws.csv", parse_dates=['Month'], index_col = 'Month')
trend_loblaws.index.name = 'date'
trend_loblaws.columns = ['trend_loblaws']
print(trend_loblaws.head())
print(trend_loblaws.tail())

trend_loblaws
date                     
2004-01-01  27           
2004-02-01  27           
2004-03-01  24           
2004-04-01  26           
2004-05-01  29           
            trend_loblaws
date                     
2019-06-01  37           
2019-07-01  37           
2019-08-01  37           
2019-09-01  35           
2019-10-01  34           


In [21]:
trend_loblaws.to_csv("trend_loblaws.csv")

In [22]:
trend_stock = pd.read_csv("trend_stock.csv", parse_dates=['Month'], index_col = 'Month')
trend_stock.index.name = 'date'
trend_stock.columns = ['trend_stock']
print(trend_stock.head())
print(trend_stock.tail())

trend_stock
date                   
2004-01-01  17         
2004-02-01  17         
2004-03-01  15         
2004-04-01  14         
2004-05-01  13         
            trend_stock
date                   
2019-06-01  65         
2019-07-01  68         
2019-08-01  74         
2019-09-01  63         
2019-10-01  64         


In [23]:
trend_stock.to_csv("trend_stock.csv")